In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd drive/project_data/classifier_august/

/content/drive/project_data/classifier_august


In [0]:
ls

49*6lable.npy         pascalX_val.npy
Classifier10epoch.h5  pascaly.npy
Classifier11epoch.h5  pascaly_val.npy
Classifier15epoch.h5  vgg16_weights_tf_dim_ordering_tf_kernels.h5
Classifier20epoch.h5  X_yolo.npy
Classifier5epoch.h5   Y9*9*5*20.npy
pascalX.npy           Y_yolo.npy


# Imports and load data

In [0]:
import keras
from keras import backend as K
import tensorflow as tf
import tensorflow
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU, Softmax
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

import numpy as np
import os
import cv2
import math

Using TensorFlow backend.


In [0]:
X = np.load('X_yolo.npy')
Y = np.load('Y9*9*5*20.npy')

print(X.shape)
print(Y.shape)

(21503, 300, 300, 3)
(21503, 9, 9, 5, 20)


In [0]:
# create Y_true (shape = 9*9*5*20)
def closs(y_true, y_pred):
  return K.categorical_crossentropy(y, y_pred)

In [0]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
input_image = Input(shape=(300, 300, 3))

#############################################################
####### VGG16 Model  ########################################
#############################################################
# Block 1
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1', trainable=False)(input_image)
x = BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv1', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv2', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv3', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv1', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv2', trainable=False)(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv3', trainable=False)(x)
x = BatchNormalization()(x)

# remove the below line of pooling in detector model(when concatanating)
#x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)



#############################################################
######## Skip connection  ###################################
#############################################################

skip_connection = x

x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
#############################################################
######## Extra Layers  ######################################
#############################################################

# Block 6
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block6_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block6_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='softmax',
                  padding='same',
                  name='block6_conv3')(x)
x = BatchNormalization()(x)

# Block 7
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block7_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block7_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='softmax',
                  padding='same',
                  name='block7_conv3')(x)
x = BatchNormalization()(x)


# Block 8
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', activation='softmax', use_bias=False)(skip_connection)
skip_connection = BatchNormalization()(skip_connection)
#skip_connection = Softmax(axis=-1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)


# Merge
x = concatenate([skip_connection, x])



model = Model(input_image, x)

# Block 9
x = Conv2D(5*20, (1,1), strides=(1,1), padding='same', name='conv_last')(x) 
x = Reshape((9, 9, 5, 20))(x)



model = Model(input_image, x)


In [0]:
model.load_weights('Classifier20epoch.h5',by_name=True)

In [0]:
optimizer = SGD(lr=0.001, decay=0.0, momentum=0.9)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])



In [0]:
model.fit(X, Y, batch_size=24, epochs=5)

Epoch 1/5
 3912/21503 [====>.........................] - ETA: 17:44 - loss: 0.3179 - acc: 0.0428

KeyboardInterrupt: ignored

In [0]:
model.save_weights('ExtendedClassifier5epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('ExtendedClassifier10epoch.h5')

In [0]:
    y = np.zeros((21503,9,9,5,20), dtype=float)

    for b in range(21503):
      for k in range(49):

        cl = int(Y[b,k,0] - 1)
        x_ = (Y[b,k,1] * 300)
        y_ = (Y[b,k,2] * 300)

        cell_x = (int(x_ / 33.33))
        cell_y = (int(y_ / 33.33))

        y[b,cell_x,cell_y,0,cl] = 1
        y[b,cell_x,cell_y,1,cl] = 1
        y[b,cell_x,cell_y,2,cl] = 1
        y[b,cell_x,cell_y,3,cl] = 1
        y[b,cell_x,cell_y,4,cl] = 1

In [0]:
np.save('Y9*9*5*20.npy', y)

In [0]:
y.shape

(21503, 9, 9, 5, 20)

In [0]:
import numpy as np

In [0]:
cd drive/project_data/30Th august

/content/drive/project_data/30Th august


In [0]:
ls

ssd2007_train_labels.npy  TEST+MODEL+SSD+300.ipynb


In [0]:
y = np.load('ssd2007_train_labels.npy')

In [0]:
y[0,0]

array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -1.33333333, -1.33333333,        -inf,        -inf,
        4.        ,  4.        , 30.        , 30.        ,  0.1       ,
        0.1       ,  0.2       ,  0.2       ])

In [0]:
y[0,1]

array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        4.        ,  4.        , 42.42640687, 42.42640687,  0.1       ,
        0.1       ,  0.2       ,  0.2       ])

In [0]:
y.shape

(2501, 8732, 33)